In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import random

%matplotlib inline

In [ ]:
import sys

sys.path.append("./")
sys.path.append("../code/")
sys.path.append("./code/")

import time

In [ ]:
from typing import List, Tuple, Union

In [ ]:
from loguru import logger

# Data Loader

In [ ]:
from bps_numerical.preprocessing import merge_gene_phenotype, standardize_gene_data

In [ ]:
CSV_GENE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/gen.csv"

CSV_PHENOTYPE = "/Users/nishparadox/dev/uah/nasa-impact/gene-experiments/data/OneDrive_1_3-21-2022/meta.csv"

In [ ]:
# df_merged = merge_gene_phenotype(standardize_gene_data(CSV_GENE), CSV_PHENOTYPE)

In [ ]:
df_genes = standardize_gene_data(CSV_GENE)

In [ ]:
df_genes.head()

In [ ]:
samples = df_genes.pop("Sample")

In [ ]:
df_genes = df_genes.astype(float)

In [ ]:
df_genes.iloc[0].dtype

# Feature Selection

In [ ]:
from bps_numerical.clustering import CorrelationClusterer
from bps_numerical.feature_selection import FirstFeatureSelector, KRandomizedFeatureSelector

In [ ]:
clusterer = CorrelationClusterer(
    list(df_genes.columns),
    cutoff_threshold=0.3,
    debug=False
)

In [ ]:
fs = FirstFeatureSelector(clusterer=clusterer)
# fs = KRandomizedFeatureSelector(clusterer=clusterer, k_features=2)

In [ ]:
cols_genes = fs.select_features(df_genes)

In [ ]:
# cols_genes = list(df_genes.columns)

In [ ]:
len(cols_genes)

In [ ]:
cols_genes[:10]

In [ ]:
# len(df_genes.columns)

# Data Prep

In [ ]:
df_merged = merge_gene_phenotype(
    pd.concat([samples, df_genes[cols_genes]], axis=1),
    CSV_PHENOTYPE,
    "Sample",
)

In [ ]:
df_merged.shape

In [ ]:
df_merged.head()

In [ ]:
len(cols_genes)

# Ranker

To further narrow down the gene space, we could train N different isolated classifiers for a specific
phenotype and then use those genes for training only that phenotype later.

In [ ]:
from bps_numerical.classification.feature_scorers import GeneRanker

In [ ]:
ranker = GeneRanker(
    cols_genes, 
    phenotype="condition",
    n_runs = 5,
    debug=True,
)

In [ ]:
features = ranker.get_features(df_merged, test_size=0.2, top_k=500, ignore_zeros=True, normalize=True)

In [ ]:
len(features)

In [ ]:
fts, _ = zip(*features)
fts

# Trainer

In [ ]:
from bps_numerical.classification import SinglePhenotypeClassifier, MultiPhenotypeIsolatedClassifier

In [ ]:
# single phenotype

# model = xgboost.XGBClassifier()
clf = SinglePhenotypeClassifier(
    cols_genes=cols_genes,
    phenotype="condition",
#     model = model
)

In [ ]:
# tracker_single = clf.train(df_merged)

In [ ]:
# tracker_single

# Multiple phenotypes

In [ ]:
clf_condition = SinglePhenotypeClassifier(cols_genes, "condition")
clf_strain = SinglePhenotypeClassifier(cols_genes, "strain")
clf_gender = SinglePhenotypeClassifier(cols_genes, "gender")
clf_mission = SinglePhenotypeClassifier(cols_genes, "mission")
clf_animal_return = SinglePhenotypeClassifier(cols_genes, "animalreturn")
trainer = MultiPhenotypeIsolatedClassifier(
    cols_genes=cols_genes,
    classifiers=[
        clf_condition,
        clf_strain,
        clf_gender,
        clf_mission,
        clf_animal_return
    ],
    debug=True,
)

In [ ]:
tracker_multi = trainer.train(df_merged)

In [ ]:
tracker_multi

In [ ]:
tracker_multi["animalreturn"]

# Feature Scorer

In [ ]:
from bps_numerical.classification.feature_scorers import PhenotypeFeatureScorer

In [ ]:
for clf in trainer.classifiers:
    print(clf.phenotype, len(PhenotypeFeatureScorer(clf).get_features(top_k=500, ignore_zeros=True)))

In [ ]:
PhenotypeFeatureScorer(clf).get_features(top_k=500, ignore_zeros=True, normalize=True)

In [ ]:
len(PhenotypeFeatureScorer(trainer).get_features(top_k=500, ignore_zeros=False))

In [ ]:
# if we ignore 0-score features
len(PhenotypeFeatureScorer(*trainer.classifiers).get_features(top_k=500, ignore_zeros=True))

In [ ]:
list(map(lambda f: f[0], PhenotypeFeatureScorer(clf_condition, clf_strain).get_features(top_k=500, ignore_zeros=True, normalize=True)))

### permutation

In [ ]:
import itertools

In [ ]:
def compute_permuted_scores(*classifiers, ignore_zeros: bool = True, top_k: int = 500):
    def _powerset(items):
        for sl in itertools.product(*[[[], [i]] for i in items]):
            yield {j for i in sl for j in i}
    
    res = {}
    for objs in _powerset(classifiers):
        if len(objs) < 2:
            continue
        labels = tuple(map(lambda clf: clf.phenotype, objs))
        res[labels] = PhenotypeFeatureScorer(*objs).get_features(top_k=top_k, ignore_zeros=True, normalize=True)
    return res

In [ ]:
permuted_ =  compute_permuted_scores(*trainer.classifiers, top_k=1000)

In [ ]:
dict(map(lambda p: (p[0], (len(p[1]), p[1])), permuted_.items()))

In [ ]:
dict(map(lambda p: (p[0], len(p[1])), permuted_.items()))

# Plot Top features

In [ ]:
def plot_features(features: List[Tuple[str, float]], view_slicer:int = 75):
    df_top_k = pd.DataFrame(features[:view_slicer], columns=["gene", "importance"])
    fig = px.bar(
        df_top_k,
        x="importance",
        y="gene",
        title=f"{view_slicer} features",
        orientation="h",
        height=1600,
        width=1000,
    #     text_auto=True,
    )
    # fig.update_traces(width=3)
    fig.update_layout(yaxis = dict(tickfont = dict(size=7)))
    fig.show()

In [ ]:
plot_features(
    PhenotypeFeatureScorer(clf_mission, clf_strain).get_features(top_k=500, ignore_zeros=True, normalize=True)
)